<!--<badge>--><a href="https://colab.research.google.com/github/huggingface/workshops/blob/main/machine-learning-tokyo/03-gradio-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

# Creating a Transformers demo with Gradio
> exploring Huggingface Transformers library in MLt workshop part 3

- toc: true
- badges: true
- comments: true
- categories: [jupyter]
- image/chart-preview


# Creating a Transformers demo with Gradio

## References

* https://huggingface.co/blog/gradio-spaces
* https://huggingface.co/blog/gradio

In [ ]:
# !pip install transformers gradio sentencepiece

## Example 1: Using the Transformers pipeline

In [ ]:
import gradio as gr
from transformers import pipeline

In [ ]:
pipe = pipeline("text-classification", model="lewtun/xlm-roberta-base-finetuned-marc-en")

In [ ]:
pipe("The Lord of the Rings is waaay too long to read!!")

In [ ]:
label2emoji = {"terrible": "💩", "poor": "😾", "ok": "🐱", "good": "😺", "great": "😻"}

def predict(text):
    preds = pipe(text)[0]
    return label2emoji[preds["label"]], round(preds["score"], 5)

predict("I love this soccer ball")

In [ ]:
gradio_ui = gr.Interface(
    fn=predict,
    title="Predicting review scores from customer reviews",
    description="Enter some review text about an Amazon product and check what the model predicts for it's star rating.",
    inputs=[
        gr.inputs.Textbox(lines=5, label="Paste some text here"),
    ],
    outputs=[
        gr.outputs.Textbox(label="Label"),
        gr.outputs.Textbox(label="Score"),
    ],
    examples=[
        ["My favourite book is Cryptonomicon!"], ["私の好きな本は「クリプトノミコン」です"]
    ],
)

gradio_ui.launch(debug=True)

## Example 2: Using the Inference API from the Hugging Face Hub

In [ ]:
from huggingface_hub import InferenceApi

text = "My favourite book is Cryptonomicon!"
inference = InferenceApi("lewtun/xlm-roberta-base-finetuned-marc-en")
preds = inference(inputs=text)
preds[0]

In [ ]:
sorted_preds = sorted(preds[0], key=lambda d: d['score'], reverse=True) 
sorted_preds

In [ ]:
def inference_predict(text):
    inference = InferenceApi("lewtun/xlm-roberta-base-finetuned-marc-en")
    preds = inference(inputs=text)
    sorted_preds = sorted(preds[0], key=lambda d: d['score'], reverse=True)[0]
    return label2emoji[sorted_preds["label"]], round(sorted_preds["score"], 5)

In [ ]:
inference_predict(text)

In [ ]:
gradio_ui = gr.Interface.load(
    name="lewtun/xlm-roberta-base-finetuned-marc",
    src="huggingface",
    fn=inference_predict,
    title="Review analysis",
    description="Enter some text and check if model detects it's star rating.",
    inputs=[
        gr.inputs.Textbox(lines=5, label="Paste some text here"),
    ],
    outputs=[
        gr.outputs.Textbox(label="Label"),
        gr.outputs.Textbox(label="Score"),
    ],
    examples=[
        ["My favourite book is Cryptonomicon!"], ["私の好きな本は「クリプトノミコン」です"]
    ],
)

gradio_ui.launch(debug=True)